In [ ]:
import os
import pickle
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer


df = pd.read_json("ScaleQM+_test.json")
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
llm = LLM(model="CoT-Bridge", tensor_parallel_size=4, gpu_memory_utilization=0.85)
tokenizer = AutoTokenizer.from_pretrained("CoT-Bridge")
texts = []
for i in range(len(df)):
    system1 = df.iloc[i]["messages"][0]["content"]
    prompt1 = df.iloc[i]["messages"][1]["content"]
    new_prompt = f"""System: {system1}\nHuman: {prompt1}\nAssistant:"""
    texts.append(new_prompt)

outputs = llm.generate(
    texts,
    SamplingParams(
    temperature=0,
    max_tokens=1024,
    skip_special_tokens=True
)
)

results = []
for i in range(len(outputs)):
    results.append(outputs[i].outputs[0].text)

with open('results-ours.pkl', 'wb') as f:
    pickle.dump(results, f)